In [ ]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten
from keras.layers import Dropout, Dense
from keras import backend

from sklearn.datasets import load_files
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import json
import os

In [ ]:
def show_image(np_array):
    %matplotlib inline
    plt.figure()
    plt.imshow(np_array, cmap='gray')

# Loading the images

    x_eq - arrays of pixes of eq's
    x_an - arrays of pixes of an's

In [ ]:
eq_files = []
for root, _, files in os.walk(os.path.normpath('./pics/data')):
    for file in files:
        eq_files.append(os.path.join(root, file))

eq_files = sorted(eq_files, key=lambda b: int(b[b.find("_")+1:b.find(".")]))

In [ ]:
an_files = []
for root, _, files in os.walk(os.path.normpath('./pics/labels')):
    for file in files:
        an_files.append(os.path.join(root, file))

an_files = sorted(an_files, key=lambda b: int(b[b.find("_")+1:b.find(".")]))

border - is 60% of data

In [ ]:
border = int(len(eq_files)*0.6)

# Data preprocessing

In [ ]:
x_eq = np.array([np.array(load_img(fname, grayscale=True)) for fname in  eq_files])
x_an = np.array([np.array(load_img(fname, grayscale=True)) for fname in  an_files])

In [ ]:
train_x_eq = x_eq[:border]
test_x_eq = x_eq[border:]
train_x_an = x_an[:border]
test_x_an = x_an[border:]


data_labels - array with eq and 1/0 if correspondent answer is correct/wrong

In [ ]:
with open("./pics/binary_labels.json") as datafile_labels:
    data_labels = json.load(datafile_labels)

In [ ]:
labels = [data_labels[i] for i in sorted(data_labels.keys(), key=lambda b: int(b[b.find("_")+1:b.find(".j")]))]

In [ ]:
train_y = labels[:border]
test_y = labels[border:]

Just to check if everything is OK with a random equation

In [ ]:
show_image(train_x_eq[23])
train_y[23]

In [ ]:
show_image(train_x_an[0])

In [ ]:
print(train_x_an.shape)
print(train_x_eq.shape)

In [ ]:
train_x = []
for i in range(len(train_x_eq)):
    train_x.append([train_x_eq[i], train_x_an[i]])
train_x = np.array(train_x)

In [ ]:
test_x = []
for i in range(len(test_x_eq)):
    test_x.append([test_x_eq[i], test_x_an[i]])
test_x = np.array(test_x)

In [ ]:
train_y = np_utils.to_categorical(train_y, 2)
test_y = np_utils.to_categorical(test_y, 2)

In [ ]:
if backend.image_data_format() != 'channels_first':
    train_x = train_x.reshape(train_x.shape[0], 28, 165, 2)
    test_x = test_x.reshape(test_x.shape[0], 28, 165, 2)
    input_shape = (28, 165, 2)

train_x = train_x.astype('float32')
train_x = train_x/255

test_x = test_x.astype('float32')
test_x = test_x/255

In [ ]:
batch_size = 32
epochs = 20

In [ ]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_first"))
model.add(Convolution2D(64, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_first"))
model.add(Dropout(0.25))
model.add(Convolution2D(128, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_first"))
model.add(Convolution2D(256, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_first"))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(84,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
print(train_x.shape,  train_y.shape)
print(test_x.shape, test_y.shape)
print(len(test_y))

In [ ]:
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(test_x, test_y))

In [ ]:
loss,accuracy = model.evaluate(test_x, test_y, verbose=1)
print("Loss ",loss)
print("Accuracy ", accuracy)

In [ ]:
loss,accuracy = model.evaluate(train_x, train_y, verbose=1)
print("Loss ",loss)
print("Accuracy ", accuracy)